# Курсовой проект: Детекция клеток малярии с использованием Faster R-CNN

Авторы: Санников Максим Витальевич, Смирнов Игорь Сергеевич


In [ ]:
# 📦 Установка библиотек
!pip install -U pip
!pip install kaggle
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/detectron2.git

In [ ]:
# 🔑 Настройка Kaggle API
import shutil
from pathlib import Path

Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
shutil.copy("kaggle.json", "/root/.kaggle/kaggle.json")
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# 📥 Загрузка датасета
!kaggle datasets download -d kmader/malaria-bounding-boxes
!unzip -qo malaria-bounding-boxes.zip -d malaria

⬆️ Далее следует блок с конвертацией в COCO, регистрацией и обучением модели (см. `train_model.py`)